# Existing Story in a new language
Here we will load a story JSON file (that contains the text), but process it into a new language

In [1]:
%load_ext autoreload
%autoreload 2
from dotenv import load_dotenv
load_dotenv()

PAY_FOR_API = True #change to True to run cells that cost money via API calls

In [2]:
from pathlib import Path
from src.config_loader import config
from PIL import Image
from src.utils import load_json, load_text_file, save_json, save_pickle, upload_to_gcs, upload_story_to_gcs
from src.generate import add_audio, add_translations
from src.story import generate_index_html
# Add the parent directory of 'src' to the Python path


setting voice override: sv-SE-Wavenet-F
setting voice override: sv-SE-Wavenet-G
FFmpeg path added to system PATH: C:\Program Files\ffmpeg-7.0-essentials_build\bin


### Add directories
story images can be re-used between languages, but audio files are language specific, so we structure the story directory story_name/language with audio files in 'language/' and images and the english JSON file in story_name dir

In [3]:
notebook_dir = Path().absolute()  # This gives src/notebooks
phrase_dir = notebook_dir.parent / "data" / "phrases" #where we store text files of phrases
story_dir = notebook_dir.parent / "outputs" / "stories" # where we store our stories

In [4]:
story_name = "swedish_adventure_in_winter_wilderness" #omit the leading story_
clean_story_name = f"story_{story_name.lower().replace(' ', '_')}"
story_path = story_dir / clean_story_name / f"{clean_story_name}.json"

story_dict = load_json(story_path)


## Generate the story files
Once you are happy with the flashcard coverage, you can:
* translate and add audio
* create the story images
* create the story album files (M4a files with synced lyrics)
* create the story HTML file using those previous files, and upload to Google Cloud Storage
* tag the flascards with the story name...this will then mean you can link to the story from within Anki (the template uses tags to auto-create hyperlinks)

In [5]:
story_dialogue_audio = add_translations(story_dict)
story_dialogue_audio = add_audio(story_dialogue_audio)

adding translations:   0%|          | 0/3 [00:00<?, ?it/s]

Beginning translation for introduction


adding translations:  33%|███▎      | 1/3 [00:02<00:05,  2.54s/it]

Translated dialogue
Beginning translation for development


adding translations:  67%|██████▋   | 2/3 [00:04<00:02,  2.24s/it]

...

Text-to-speech for dialogue done
Beginning text-to-speech for development


adding audio:  67%|██████▋   | 2/3 [00:53<00:24, 24.92s/it]

Text-to-speech for dialogue done
Beginning text-to-speech for resolution


adding audio: 100%|██████████| 3/3 [01:07<00:00, 22.57s/it]

Text-to-speech for dialogue done


In [6]:
#this has target language content in now so we save in language dir
save_pickle(data=story_dialogue_audio, file_path=story_dir / clean_story_name / config.TARGET_LANGUAGE_NAME / f"{clean_story_name}.pkl")

M4A audio files which you will be able to download and play via a media player.
They have synced lyrics which can be viewed in the Oto Music Player app

In [7]:
from src.story import create_album_files
FIRST_STORY_PART = list(story_dialogue_audio.keys())[0]
#may need to change depending on size of story made and what parts there are
album_image = Image.open(story_dir / clean_story_name / f"{clean_story_name}_{FIRST_STORY_PART}.png")
#create m4a file:
create_album_files(story_data_dict=story_dialogue_audio, cover_image=album_image, output_dir=story_dir / clean_story_name / config.TARGET_LANGUAGE_NAME, story_name=clean_story_name)

creating album:  33%|███▎      | 1/3 [00:03<00:07,  3.65s/it]

Saved M4A file track number 1


creating album:  67%|██████▋   | 2/3 [00:07<00:03,  3.85s/it]

Saved M4A file track number 2


creating album: 100%|██████████| 3/3 [00:10<00:00,  3.58s/it]

Saved M4A file track number 3


Now we generate the main html file - this wraps up the M4A files and image files within it, so it's self-contained

In [8]:
from src.story import create_html_story

create_html_story(
            story_data_dict=story_dialogue_audio,
            image_dir=story_dir / clean_story_name, #the langauge sub-folders will be picked up automatically
            story_name=clean_story_name,
        )

Preparing HTML data: 100%|██████████| 3/3 [04:29<00:00, 89.98s/it] 


HTML story created at: y:\Python Scripts\audio-language-trainer\outputs\stories\story_swedish_adventure_in_winter_wilderness\Swedish\story_swedish_adventure_in_winter_wilderness.html


Upload to a public google cloud bucket

In [9]:
html_story_path = story_dir / clean_story_name / config.TARGET_LANGUAGE_NAME / f"{clean_story_name}.html"
assert html_story_path.exists()
upload_story_to_gcs(html_file_path=html_story_path)

'https://storage.googleapis.com/audio-language-trainer-stories/swedish/story_swedish_adventure_in_winter_wilderness/story_swedish_adventure_in_winter_wilderness.html'

Update the index webpage

In [10]:
generate_index_html()
#will default to public GCS bucket
upload_to_gcs(
    file_path="../outputs/stories/index.html",
    content_type="text/html"
)

'https://storage.googleapis.com/audio-language-trainer-stories/index.html'